# Dynamics exercise

In this Python exercise, we will analyze the dynamic response of a simple elastic rectangular body under plain stress that is released from an initially static equilibrium under external traction $\boldsymbol{ \hat{t}}$ at time $t=0$:

![pure bending example](images/dynamics_bending.png)

The finite element code is not yet complete and requires some self implementation.

## Implementation

In Python, it is necessary to import packages that we want to use. Here, we include the package `femtoe` which contains many finite element helper functions that we are going to use. These functions are defined in the directory `femtoe`. Additionally, we will need some vector and tensor operations in Python. The package of choice is numpy in this case. And for creating plots, we use matplotlib.

In [ ]:
import femtoe
import numpy as np
import matplotlib.pyplot as plt

In the following, we will do the implementation of the dynamic bending problem. We will write a function that generates the mesh, evaluates the elements, assembles the global system, solves for the initial condition and runs the simulation for a certain number of timesteps. It looks like this:

In [ ]:
def solve_dynamic_elastic_beam(
    num_ele_y: int,
    num_timesteps: int,
    max_time: float,
    beta: float,
    gamma: float,
    plot: bool = False,
):
    """
    Solves the dynamic bending problem for a ginve number of elements in y-direction and parameters for the time integration scheme

    Parameters
    ----------
    num_ele_y: int
        Number of elements in y-direction

    num_timesteps: int
        Number of timesteps

    max_time: float
        The maximum time of the simulation

    beta: float
        A parameter of the Newmark integration scheme

    gamma: float
        A parameter of the Newmark integration scheme

    plot: bool
        Wheter to plot the results    
    """
    # create our mesh and the boundary conditions
    beam_width = 0.5
    problem = femtoe.pure_bending.PureBending(
        width=beam_width,
        height=0.1,
        num_ele_x=5 * num_ele_y,
        num_ele_y=num_ele_y,
        youngs_modulus=2.0e11,
        nu=0,
        rho=8000,
        plain_stress=True,
    )

    # we have a uniform traction in negative y-direction at the end of the beam
    problem.traction = femtoe.pure_bending.Traction(
        problem.mesh, lambda _: -1e8 * np.array([0, 1]))

    # Evaluate K and F
    global_stiffness_matrix = femtoe.evaluate_global_stiffness_matrix(problem)
    global_force_vector = femtoe.evaluate_global_force_vector(problem)

    # Solve initial static equilibrium
    D_n = np.linalg.solve(global_stiffness_matrix, global_force_vector)

    # compute element mass matrix
    global_mass_matrix = np.zeros((problem.num_dof, problem.num_dof))
    for cell in problem.cells:
        mass_matrix = compute_element_mass_matrix(problem, problem.nodes[cell])

        femtoe.assemble_matrix(
            global_mass_matrix,
            mass_matrix,
            cell,
            problem.num_dof_per_node
        )

    # Now, we need to apply our Dirichlet boundary condition
    dirichlet_dofs = problem.dirichlet_dofs # these dofs are fixed
    free_dofs = np.delete(np.arange(problem.num_dof), dirichlet_dofs) # these are the remaining dofs

    # apply dirichlet boundary condition to mass matrix
    global_mass_matrix = global_mass_matrix[np.ix_(free_dofs, free_dofs)]

    # compute Rayleigh damping matrix
    global_damping_matrix = 0 * global_mass_matrix + 3e-5 * global_stiffness_matrix

    # from now on, our external force should be zero
    global_force_vector = np.zeros_like(D_n)

    # compute initial conditions for velocity
    V_n = np.zeros_like(D_n)

    # Initial acceleration is given by the ODE
    A_n = np.linalg.solve(global_mass_matrix, global_force_vector-global_damping_matrix @ V_n - global_stiffness_matrix @ D_n)

    timesteps = []
    displacements = []

    # find dof-id of vertical displacement of point P
    dof_id = (
        2
        * np.argwhere(
            np.linalg.norm(problem.nodes - np.array([beam_width, 0]), axis=1) < 1e-9
        )[0, 0]
        + 1
    )
    dof_id_star = np.argwhere(np.arange(problem.num_dof)[free_dofs] == dof_id)[0, 0]

    delta_time = max_time / num_timesteps

    # create the Newmark integration scheme (we will implement it later)
    time_integrator = create_newmark_time_integrator(
        global_mass_matrix,
        global_damping_matrix,
        global_stiffness_matrix,
        beta=beta,
        gamma=gamma,
        delta_time=delta_time
    )

    # here is our timeloop
    for timestep in range(num_timesteps):
        # at every timestep, we need to do an integration step of our time integration scheme
        D_n, V_n, A_n = time_integrator(D_n, V_n, A_n, global_force_vector)

        # store the results so that we can plot them
        timesteps.append(timestep)
        displacements.append(D_n[dof_id_star])

    # we might want to plot it
    if plot:
        import matplotlib.pyplot as plt
        plt.plot(np.array(timesteps) *delta_time, displacements)
        plt.xlabel('Time')
        plt.ylabel('Displacement')
        plt.show()

    # return the displacements at the end
    return displacements[-1]

## 1.2 Exercise 1: Implementation of $\boldsymbol{m}^{(e)}$

In the snippet above, the mass matrix is not yet implemented.

First recall, the element mass matrix using the Gauss quadrature is computed as:

$$
\boldsymbol{m}^{(e)} = \sum_{g=1}^{n_\text{gp}} \rho {\boldsymbol{N}_g^{(e)}}^\mathrm{T} \boldsymbol{N}_g^{(e)} \mathrm{det}( \boldsymbol{J}_g ) w_g,
$$
where
$$
\boldsymbol{N}_g^{(e)} = \left[\begin{matrix}
    N^{1(e)} & 0 & \cdots N^{1(e)} & 0 \\
    0 & N^{1(e)} & 0 & \cdots N^{1(e)}
\end{matrix}\right]
$$

Implement the element mass matrix in the provided snippet.

In [ ]:
def compute_element_mass_matrix(problem: femtoe.pure_bending.PureBending, nodal_coordinates: np.ndarray) -> np.ndarray:
    """
    Computes the element mass matrix of an element

    Parameters
    -----------
    problem: femtoe.pure_bending.PureBending
        An object describing the problem (mesh, boundary conditions, material parameters, etc.)
        

    nodal_coordinates: np.ndarray
        The coordinates of all nodes of the element (num_nodes_per_ele x dim).

    Returns:
    --------
    element_mass_matrix: np.ndarray
        The element mass matrix for linear elasticity (num_dof_per_element x num_dof_per_element).
    """
    # initialize empty mass matrix for integration
    element_mass_matrix = np.zeros(
        (problem.num_dof_per_element, problem.num_dof_per_element)
    )

    # loop over Gauss points
    for gauss_weight, gauss_point in femtoe.gauss_integration(problem.cell_type, 4):
        # evaluate jacobian matrix
        shape_function_derivatives = femtoe.evaluate_shape_function_derivatives(
            problem.cell_type, gauss_point
        )
        jacobian = shape_function_derivatives.T @ nodal_coordinates
        det_jacobian = np.linalg.det(jacobian)

        # density
        rho = problem.rho

        # evaluate shape functions
        shape_functions = femtoe.evaluate_shape_functions(problem.cell_type, gauss_point)

        # do implementation here
        
        element_mass_matrix += 

    return element_mass_matrix

## Exercise 2: Implementation of Newmark's method

The second ingredient for a dynamics problem is a time integration method. In this case, we want to implement Newmarks method. Recall that the numerical scheme of the Newmark method is given by the following three equations:

$$
\boldsymbol{M} \boldsymbol{A}_{n+1} + \boldsymbol{C} \boldsymbol{V}_{n+1} + \boldsymbol{K} \boldsymbol{D}_{n+1} = \boldsymbol{F}_{n+1},
$$
$$
\boldsymbol{D}_{n+1} = \boldsymbol{D}_{n} + \Delta t \boldsymbol{V}_{n} + \frac{\Delta t^2}{2} \left[ (1-2\beta)\boldsymbol{A}_{n} + 2\beta\boldsymbol{A}_{n+1} \right],
$$
$$
\boldsymbol{V}_{n+1} = \boldsymbol{V}_{n} + \Delta t \left[ \gamma \boldsymbol{A}_{n} + (1-\gamma) \boldsymbol{A}_{n+1}\right].
$$

Here, the subscripts $[\cdot]_n$ and $[\cdot]_{n+1}$ refer to quantities at the beginning and at the end of the timestep, respectively. The quantities with the subscript $[\cdot]_n$ are assumed to be known from the previous timestep.

By substituting $\boldsymbol{D}_{n+1}$ and $\boldsymbol{V}_{n+1}$ in Equation (1), the unknown acceleration $\boldsymbol{A}_{n+1}$ can be computed in terms of known quantities my solving the linear system

$$
\left(\boldsymbol{M} + (1-\gamma) \Delta t \boldsymbol{C} + \beta \Delta t^2 \boldsymbol{K}\right) \boldsymbol{A}_{n+1} = \boldsymbol{F}_{n+1} - \boldsymbol{K}\boldsymbol{D}_{n} - (\boldsymbol{C}+\Delta t \boldsymbol{K}) \boldsymbol{V}_{n} - \left( \gamma \Delta t \boldsymbol{C} + (1-2\beta) \frac{\Delta t^2}{2} \boldsymbol{K} \right) \boldsymbol{A}_{n}.
$$

Then, once $\boldsymbol{A}_{n+1}$ is available, the unknown displacements and velocities, $\boldsymbol{D}_{n+1}$ and $\boldsymbol{V}_{n+1}$, are computed using the equations above.

Implement Newmark's theme in the snippet below.

In [ ]:
def create_newmark_time_integrator(
    M: np.ndarray,
    C: np.ndarray,
    K: np.ndarray,
    beta: float,
    gamma: float,
    delta_time: float,
):
    """
    This function returns a function that is doing a time-integration step with the given parameters above

    Parameters
    ----------
    M: np.ndarray
        The global mass matrix of the problem

    C: np.ndarray
        The damping matrix of the problem

    K: np.ndarray
        The stiffness matrix of the problem

    beta: float
        A parameter of Newmark's method

    gamma: float
        A parameter of Newmark's method

    delta_time: float
        The timestep size

    Returns
    -------
    do_integration_step:
        The function returns another function that performs the actual time integration step.
    """

    # Note, this function is executed only once, so we can also do expensve operations here.
    # Here, we can precompute some matrices that are expensive to evaluate but don't change over time

    def do_integration_step(
        D_n: np.ndarray,
        V_n: np.ndarray,
        A_n: np.ndarray,
        F_np: np.ndarray,
    ):
        # Note, this function here is executed on every timestep! Try to put as much expensive operations as possible in the function above.
        
        # Here, we need to do the actual time integration step. We can reuse the parameters from the function above!
        A_np = 
        D_np = 
        V_np = 

        return D_np, V_np, A_np

    # return the inner function that performs the actual time integration step
    return do_integration_step

## Verification

In the next step, we want to verify your implementation. Here, we compare the displacement at the final time with a reference solution.

In [ ]:
solution = solve_dynamic_elastic_beam(
    num_ele_y=2, num_timesteps=400, max_time=10e-3, beta=0.25, gamma=0.5, plot=True
)
if np.allclose(solution, 0.006110302146730411):
    print("It looks like your implementation is correct! 🥳🥳🥳")
else:
    print(
        "Unfortunately, your implementation still contains an error!😭 Please fix it before continuing"
    )

## Exercise 2: Analysis

A newly implemented code should always be verified. A standard practice in FE is to check if the implementation leads to the expected convergence rates. In general, both the convergence of the space and time discretizations have to be checked. But for the sake of simplicity, we will only verify here the convergence in time.

### Task 1: Trapezoidal rule

We consider the so-called trapezoidal rule, which is a particular case of Newmark's method for the parameters $\beta=0.25$ and $\gamma=0.5$. For a fixed space discretization with 2 elements vertical direction, compute the example for different numbers of timesteps. Compute the error using the displacement of the next finer time discretization as the "exact" value.

In [ ]:
max_time = 10e-3
number_of_timesteps_trap= np.array([100, 200, 400, 800, 1600])
delta_time = max_time / number_of_timesteps_trap
displacement_trap = np.zeros_like(number_of_timesteps_trap, dtype=float)

for i, num_timesteps in enumerate(number_of_timesteps_trap):
    print(f"Running simulation with {num_timesteps} timesteps")
    displacement_trap[i] = solve_dynamic_elastic_beam(
        num_ele_y=2,
        num_timesteps=num_timesteps,
        max_time=max_time,
        beta=0.25,
        gamma=0.5,
    )

displacement_error_trap = np.abs(displacement_trap[:-1] - displacement_trap[1:])

plt.loglog(delta_time[:-1], displacement_error_trap, marker="o")
plt.title("Trapezoidal rule")
plt.xlabel("Time step size")
plt.ylabel("Displacement error")

### Task 2: Centered difference rule

We consider now the so-called centered difference rule, which corresponds to the parameters $\beta=0$ and $\gamma=0.5$ of the Newmark method. Consider again the same spatial discretization (2 elements in y-direction). Again, compute the error using the next finer time discretization as the "exact" value.

Are you able to find a sensible result for all discretizations? Why?

In [ ]:
max_time = 10e-3
number_of_timesteps_cent = np.array([1600, 3200, 6400, 12800, 25600, 51200])
delta_time = max_time / number_of_timesteps_cent
displacement_cent = np.zeros_like(number_of_timesteps_cent, dtype=float)

for i, num_timesteps in enumerate(number_of_timesteps_cent):
    print(f"Running simulation with {num_timesteps} timesteps")
    displacement_cent[i] = solve_dynamic_elastic_beam(
        num_ele_y=2,
        num_timesteps=num_timesteps,
        max_time=max_time,
        beta=0.0,
        gamma=0.5,
    )

displacement_error_cent = np.abs(displacement_cent[:-1] - displacement_cent[1:])

plt.loglog(delta_time[:-1], displacement_error_cent, marker="o")
plt.title("Centered difference rule")
plt.xlabel("Time step size")
plt.ylabel("Displacement error")

### Task 3: Very fine reference solution

Now, we consider a reference solution using a **fine** discretization both in space and time. We use
8 elements in the vertical direction for the space discretization and 102400 timesteps. The resulting displacement is $-0.007517103923916218$. We use
this solution to recompute the error from Task 2.

Are the results consistent with the expected approximation order of the centered difference rule? If not, give an explanation of this behavior.

In [ ]:
displacement_reference = -0.007517103923916218

displacement_cent_exact = np.abs(displacement_cent - displacement_reference)

plt.loglog(delta_time[2:], displacement_cent_exact[2:], marker="o")
plt.title("Centered difference rule with reference solution")
plt.xlabel("Time step size")
plt.ylabel("Displacement error")